In [1]:
import sys
print(sys.executable)

/Users/michaelgathara/Documents/Projects/gpt/.venv/bin/python


In [2]:
import sys
import os
import json

# Get the parent directory of the current notebook
sys.path.append(os.path.abspath(".."))

In [3]:
import torch
import multiprocessing

from data import get_wikitext_data, get_fineweb_data, load_data
from tokenization import get_tiktoken_tokenizer

In [4]:
batch_size = 64  # Kept the same; could be adjusted based on hardware
block_size = 1024  # GPT-2 uses a context length of 1024 tokens
max_iters = 50000  # More iterations needed for larger models
eval_interval = 1000  # Increase since more iterations are done
learning_rate = 5e-5  # GPT-2 uses a lower learning rate
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 500  # More frequent evaluation for stability

n_embd = 768  # GPT-2 uses 768 for the small version, 1024 for medium, 1280 for large, 1600 for XL
n_head = 12  # GPT-2 uses 12 attention heads
n_layer = 12  # GPT-2 has 12 transformer blocks in the small version
dropout = 0.1  # GPT-2 uses 0.1 dropout for better generalization
print(device)

cpu


In [5]:
DATA_PATH = "/workspace/GPT/data/tiktoken_tokenized_wikitext"
num_cores = multiprocessing.cpu_count()

tokenizer = get_tiktoken_tokenizer()
dataset = get_wikitext_data()
print(tokenizer)
print(dataset)

/Users/michaelgathara/Documents/Projects/gpt/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<Encoding 'gpt2'>
DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 1801350
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})


In [6]:
tokenized_dataset = dataset.map(
    lambda x: {"tokens": [tokenizer.encode(text) for text in x["text"]]},
    batched=True,
    num_proc=num_cores,
    remove_columns=["text"]
)
print(tokenized_dataset)
tokenized_dataset = tokenized_dataset.filter(lambda x: len(x["tokens"]) > 0)
print(tokenized_dataset["train"][0])

DatasetDict({
    test: Dataset({
        features: ['tokens'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['tokens'],
        num_rows: 1801350
    })
    validation: Dataset({
        features: ['tokens'],
        num_rows: 3760
    })
})
{'tokens': [796, 569, 18354, 7496, 17740, 6711, 796, 220, 198]}
